In [29]:
from tensorflow.examples.tutorials.mnist import input_data

In [30]:
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
sess = tf.InteractiveSession()

In [20]:
def weight_variable(shape):

    """
    给权重制造随机噪声，打破完全对称，如一个截断的正态分布噪声
    """
    initial = tf.truncated_normal(shape,stddev=0.1)
    
    return tf.Variable(initial)


def bias_variable(shape):
    """
    设置为0.1，避免死亡节点
    """
    initial = tf.constant(0.1,shape=shape)
    
    return tf.Variable(initial)

In [21]:
def conv2d(x,W):
    """
    W是卷积的参数，eg[5,5,1,32] 第一第二表示卷积核的尺寸，第三表示channel个数，第四表示卷积核数量，就是
    这个卷积层会提取到多少类的特征
    
    strides 表示步长，这里设置为不遗漏第画过图片的每一个点
    
    padding='SAME' 让卷积的输出和输入保持同样的尺寸
    
    """
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
    
def max_pool_2x2(x):
    """
    以横竖两个方向为2为步长，把一个2x2图像块降为1x1的像素
    
    最大池化能保持最显著的特征
    """
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [22]:
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])
#-1表示样本数量不固定，最后的1表示颜色通道数量
x_image = tf.reshape(x,[-1,28,28,1])

In [23]:
#第一个卷积层

W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [25]:
#第二个卷积层

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [26]:
#经过两次2x2的池化，边长只有原来1/4，图片尺寸从28变为7，因为第二个卷积核数量为64，所以输出tensor尺寸为7x7x64
#链接一个全连接层，隐含节点为1024
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

In [27]:
#减轻过拟合，用dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

In [11]:
#dropout输出连接一个softmax层

In [28]:
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

In [29]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [30]:
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [32]:
tf.global_variables_initializer().run(session=sess)
for i in range(1000):
    batch = mnist.train.next_batch(50)
    if(i%100==0):
        train_accuracy = accuracy.eval(session=sess,feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print("step %d,training accuracy %g"%(i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

step 0,training accuracy 0.16
step 100,training accuracy 0.88
step 200,training accuracy 0.9
step 300,training accuracy 0.9
step 400,training accuracy 0.96
step 500,training accuracy 0.94
step 600,training accuracy 0.96
step 700,training accuracy 0.98
step 800,training accuracy 0.98
step 900,training accuracy 0.98


In [34]:
print("test accuracy %g"%accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))

test accuracy 0.9628


In [ ]:
#以下是CIFAR10

In [104]:
import cifar10,cifar10_input
data_dir = os.getcwd()
max_steps = 3000
batch_size = 128

images_train,labels_train = cifar10_input.distorted_inputs(data_dir=data_dir,batch_size=batch_size)

images_test,labels_test = cifar10_input.inputs(eval_data=True,data_dir=data_dir,batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [12]:


def variable_with_weight_loss(shape,stddev,wl):
    """
    var 是weight,用multiply让L2loss乘以wl，得到最后的weight loss
    然后使用add_to_collection把weight loss存到一个collection当中 
    """

    var = tf.Variable(tf.truncated_normal(shape,stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),wl,name='weight_loss')
        tf.add_to_collection('losses',weight_loss)

    return var


ValueError: setting an array element with a sequence.

In [93]:
image_holder = tf.placeholder(tf.float32,[batch_size,24,24,3])
label_holder = tf.placeholder(tf.int32,[batch_size])

In [94]:
#第一层卷积层

weight1 = variable_with_weight_loss(shape=[5,5,3,64],stddev=0.05,wl=0.0)
kernel1 = tf.nn.conv2d(image_holder,weight1,strides=[1,1,1,1],padding='SAME')
bias1 = tf.Variable(tf.constant(0.0,shape=[64]))
activ1 = tf.nn.relu(tf.nn.bias_add(kernel1,bias1))
pool1 = tf.nn.max_pool(activ1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
norm1 = tf.nn.lrn(pool1,4,bias=1.0,alpha=0.001/9.0,beta=0.75)


In [95]:
#第二层卷积层


weight2 = variable_with_weight_loss(shape=[5,5,64,64],stddev=0.05,wl=0.0)
kernel2 = tf.nn.conv2d(norm1,weight2,strides=[1,1,1,1],padding='SAME')
bias2 = tf.Variable(tf.constant(0.1,shape=[64]))
activ2 = tf.nn.relu(tf.nn.bias_add(kernel2,bias2))
norm2 = tf.nn.lrn(activ2,4,bias=1.0,alpha=0.001/9.0,beta=0.75)
pool2 = tf.nn.max_pool(norm2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')




In [96]:
# FC层

#要把前面的输出结果全部flatten，使用tf.reshape函数使得每个样本变成一维向量

reshape = tf.reshape(pool2,[batch_size,-1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim,384],stddev=0.04,wl=0.004)
bias3 = tf.Variable(tf.constant(0.1,shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bias3)

In [97]:
weight4 = variable_with_weight_loss(shape=[384,192],stddev=0.04,wl=0.004)
bias4 = tf.Variable(tf.constant(0.1,shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3,weight4)+bias4)

In [98]:
weight5 = variable_with_weight_loss(shape=[192,10],stddev=1/192.0,wl=0.0)
bias5 = tf.Variable(tf.constant(0.0,shape=[10]))
logits = tf.add(tf.matmul(local4,weight5),bias5)

In [99]:

def loss(logits,labels):
    """
    把正则化项的loss加上交叉熵的loss
    """
    labels = tf.cast(labels,tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels,name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    
    return tf.add_n(tf.get_collection('losses'),name='total_loss')


In [100]:
loss = loss(logits,label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
#求出结果中top k 的准确率，默认使用top 1
top_k_op = tf.nn.in_top_k(logits,label_holder,1)

In [101]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()


/home/alex/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [102]:
tf.train.start_queue_runners()

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 139796302907136)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139796276426496)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139796268033792)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139796259641088)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139796251248384)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139796242855680)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 139796234462976)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [ ]:
import time
for step in range(50):
    start_time = time.time()
    image_batch,label_batch = sess.run([images_train,labels_train])
    _,loss_value = sess.run([train_op,loss],feed_dict={image_holder:image_batch,label_holder:label_batch})
    duration = time.time()-start_time

    examples_per_sec = batch_size/duration
    sec_per_batch =float(duration)
        
    print("step %d loss=%.2f"%(step,loss_value))
        
        
        
    